In [1]:
import string
import re
import random
import collections
import copy

from enum import Enum
Direction = Enum('Direction', 'left right up down upleft upright downleft downright')
    
delta = {Direction.left: (0, -1),Direction.right: (0, 1), 
         Direction.up: (-1, 0), Direction.down: (1, 0), 
         Direction.upleft: (-1, -1), Direction.upright: (-1, 1), 
         Direction.downleft: (1, -1), Direction.downright: (1, 1)}

cat = ''.join
wcat = ' '.join
lcat = '\n'.join

In [2]:
# all_words = [w.strip() for w in open('/usr/share/dict/british-english').readlines()
#             if all(c in string.ascii_lowercase for c in w.strip())]
# words = [w for w in all_words
#          if not any(w in w2 for w2 in all_words if w != w2)]
# open('wordsearch-words', 'w').write(lcat(words))

In [3]:
# ws_words = [w.strip() for w in open('wordsearch-words').readlines()
#             if all(c in string.ascii_lowercase for c in w.strip())]
# ws_words[:10]

In [4]:
ws_words = [w.strip() for w in open('/usr/share/dict/british-english').readlines()
            if all(c in string.ascii_lowercase for c in w.strip())]

In [5]:
def empty_grid(w, h):
    return [['.' for c in range(w)] for r in range(h)]

In [6]:
def show_grid(grid):
    return lcat(cat(r) for r in grid)

In [7]:
grid = empty_grid(10, 10)
print(show_grid(grid))

..........
..........
..........
..........
..........
..........
..........
..........
..........
..........


In [8]:
def indices(grid, r, c, l, d):
    dr, dc = delta[d]
    w = len(grid[0])
    h = len(grid)
    inds = [(r + i * dr, c + i * dc) for i in range(l)]
    return [(i, j) for i, j in inds
           if i >= 0
           if j >= 0
           if i < h
           if j < w]

In [9]:
def gslice(grid, r, c, l, d):
    return [grid[i][j] for i, j in indices(grid, r, c, l, d)]

In [10]:
def set_grid(grid, r, c, d, word):
    for (i, j), l in zip(indices(grid, r, c, len(word), d), word):
        grid[i][j] = l
    return grid

In [11]:
set_grid(grid, 2, 3, Direction.downright, 'testword')
print(show_grid(grid))

..........
..........
...t......
....e.....
.....s....
......t...
.......w..
........o.
.........r
..........


In [12]:
cat(gslice(grid, 3, 2, 15, Direction.right))

'..e.....'

In [13]:
re.match(cat(gslice(grid, 3, 2, 4, Direction.right)), 'keen')

<_sre.SRE_Match object; span=(0, 4), match='keen'>

In [14]:
re.match(cat(gslice(grid, 3, 2, 3, Direction.right)), 'keen')

<_sre.SRE_Match object; span=(0, 3), match='kee'>

In [15]:
re.fullmatch(cat(gslice(grid, 3, 2, 3, Direction.right)), 'keen')

In [16]:
re.match(cat(gslice(grid, 3, 2, 4, Direction.right)), 'kine')

In [17]:
def could_add(grid, r, c, d, word):
    s = gslice(grid, r, c, len(word), d)
    return re.fullmatch(cat(s), word)

In [18]:
could_add(grid, 3, 2, Direction.right, 'keen')

<_sre.SRE_Match object; span=(0, 4), match='keen'>

In [19]:
could_add(grid, 3, 2, Direction.right, 'kine')

In [20]:
random.choice(list(Direction))

<Direction.right: 2>

In [58]:
def fill_grid(grid, words, word_count, max_attempts=10000):
    attempts = 0
    added_words = []
    w = len(grid[0])
    h = len(grid)
    while len(added_words) < word_count and attempts < max_attempts:
        attempts += 1
        r = random.randrange(w)
        c = random.randrange(h)
        word = random.choice(words)
        d = random.choice(list(Direction))
        if len(word) >=4 and not any(word in w2 for w2 in added_words) and could_add(grid, r, c, d, word):
            set_grid(grid, r, c, d, word)
            added_words += [word]
            attempts = 0
    return grid, added_words

In [22]:
g = empty_grid(20, 20)
g, ws = fill_grid(g, ws_words, 40)
len(ws)

40

In [23]:
print(show_grid(g))
print(len(ws), 'words added')
print(wcat(ws))

..pouchexecrate.....
.hazardous....t.wive
...sniradnam.aselcnu
s.weeklies.rnb......
e..lamenessse.o.....
i.tsallab..s.a.i....
tslim.......f.c.l...
iwheelbase...f.tges.
ed....llabhgihinirr.
dw.limbs..nj.bitev.s
te.wiltediu.ructs.e.
elsombretv.oqes..a..
ll..e..te.iela....m.
ie.a..un.lheleiretoc
ord..bi.scsp...kiths
ts.malcentilitren...
..i.e.sexetrov.stolb
.r.s...ruof....htrof
bgnihsirevopmi.....c
....graciousness....
40 words added
lameness impoverishing plasters wilted toilet forth coterie hazardous abutting chequing weeklies sombre execrate mastiffs boilers uncles centilitre mandarins wheelbase graciousness vortexes dwellers ballast limbs four tans highball wive broils beads mils reactive select deities shtik juveniles blots pouch brim coon


In [24]:
def present(grid, word):
    w = len(grid[0])
    h = len(grid)
    for r in range(h):
        for c in range(w):
            for d in Direction:
                if cat(gslice(grid, r, c, len(word), d)) == word:
                    return True, r, c, d
    return False, 0, 0, list(Direction)[0]

In [25]:
for w in ws:
    print(w, present(g, w))

lameness (True, 4, 3, <Direction.right: 2>)
impoverishing (True, 18, 13, <Direction.left: 1>)
plasters (True, 14, 11, <Direction.upright: 6>)
wilted (True, 10, 3, <Direction.right: 2>)
toilet (True, 15, 0, <Direction.up: 3>)
forth (True, 17, 19, <Direction.left: 1>)
coterie (True, 13, 19, <Direction.left: 1>)
hazardous (True, 1, 1, <Direction.right: 2>)
abutting (True, 15, 4, <Direction.upright: 6>)
chequing (True, 14, 9, <Direction.upright: 6>)
weeklies (True, 3, 2, <Direction.right: 2>)
sombre (True, 11, 2, <Direction.right: 2>)
execrate (True, 0, 7, <Direction.right: 2>)
mastiffs (True, 12, 18, <Direction.upleft: 5>)
boilers (True, 3, 13, <Direction.downright: 8>)
uncles (True, 2, 19, <Direction.left: 1>)
centilitre (True, 15, 6, <Direction.right: 2>)
mandarins (True, 2, 11, <Direction.left: 1>)
wheelbase (True, 7, 1, <Direction.right: 2>)
graciousness (True, 19, 4, <Direction.right: 2>)
vortexes (True, 16, 13, <Direction.left: 1>)
dwellers (True, 8, 1, <Direction.down: 4>)
ballast 

In [47]:
def interesting(grid, words, words_limit=40, direction_slack=1):
    dirs = set(present(grid, w)[3] for w in words)
    return len(words) > words_limit and len(dirs) + direction_slack >= len(delta)

In [48]:
interesting(g, ws)

True

In [51]:
def interesting_grid(width=20, height=20, words_limit=40, direction_slack=1):
    boring = True
    while boring:
        grid = empty_grid(width, height)
        grid, words = fill_grid(grid, ws_words, 80)
        boring = not interesting(grid, words, words_limit=words_limit, direction_slack=direction_slack)
    return grid, words

In [52]:
g, ws = interesting_grid()
print(show_grid(g))
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
print(wcat(ws))

dexawwt....ybossm..t
snaprhlukulele..o.er
...erioteb.hrevordla
..sivc..pr.tdekcahkp
atve..h.oiselapcy.cd
nesrehpargohtill.pue
n.leaveyis.smuguhhrt
eyhsifnt.r...aeesetc
xverbseebasedlritl.a
.r..k.wie..mdonaiaeg
..ac.nsof.admeeulsoo
g.opo.cmanaotpqiinmp
nc.us.a.lpnteaerdkiu
ionjoys.leirinioicns
wnkcent.skeracllkase
oab..erusopxeao.antt
dureifidimuhed.nskea
aga...capitols..scrl
h.v.sandblaster..i.i
s.e..sdratoelhitsn.d
61 words added;  8 directions
newscast kittenish cocks lithographers truckle leotards they exposure dehumidifier sandblaster alien paddle shadowing gondola wrest joys minster pales chairs fishy capitols based gums pheromones saki moiety waxed guano thriven dilate moray icons adman ukulele hacked rope rise clue acted nicknack spar verbs boss annex neck repeater befalls drover leave pans brave brigs opus live noun tail riot care hits quilt part


In [30]:
def datafile(name, sep='\t'):
    """Read key,value pairs from file.
    """
    with open(name) as f:
        for line in f:
            splits = line.split(sep)
            yield [splits[0], int(splits[1])]

In [31]:
def normalise(frequencies):
    """Scale a set of frequencies so they sum to one
    
    >>> sorted(normalise({1: 1, 2: 0}).items())
    [(1, 1.0), (2, 0.0)]
    >>> sorted(normalise({1: 1, 2: 1}).items())
    [(1, 0.5), (2, 0.5)]
    >>> sorted(normalise({1: 1, 2: 1, 3: 1}).items()) # doctest: +ELLIPSIS
    [(1, 0.333...), (2, 0.333...), (3, 0.333...)]
    >>> sorted(normalise({1: 1, 2: 2, 3: 1}).items())
    [(1, 0.25), (2, 0.5), (3, 0.25)]
    """
    length = sum(f for f in frequencies.values())
    return collections.defaultdict(int, ((k, v / length) 
        for (k, v) in frequencies.items()))


In [32]:
english_counts = collections.Counter(dict(datafile('count_1l.txt')))
normalised_english_counts = normalise(english_counts)

In [33]:
wordsearch_counts = collections.Counter(cat(ws_words))
normalised_wordsearch_counts = normalise(wordsearch_counts)

In [34]:
normalised_wordsearch_counts = normalise(collections.Counter(normalised_wordsearch_counts) + collections.Counter({l: 0.05 for l in string.ascii_lowercase}))

In [35]:
def weighted_choice(d):
    """Generate random item from a dictionary of item counts
    """
    target = random.uniform(0, sum(d.values()))
    cuml = 0.0
    for (l, p) in d.items():
        cuml += p
        if cuml > target:
            return l
    return None

def random_english_letter():
    """Generate a random letter based on English letter counts
    """
    return weighted_choice(normalised_english_counts)

def random_wordsearch_letter():
    """Generate a random letter based on wordsearch letter counts
    """
    return weighted_choice(normalised_wordsearch_counts)

In [36]:
cat(sorted(random_english_letter() for i in range(100)))

'aaaaaabbccddeeeeeeeeeeeffggghhhhhhiiiiiiiiillllmmnnnnnnnooooooooooppppqrrrrrrrssstttttttttuuwyyyyyyz'

In [37]:
cat(sorted(random_wordsearch_letter() for i in range(100)))

'aaaaaaaabbbccccddeeeeeeeeggghhhiiiiiiiijjkkllllnnnnoooooooooopppqqrrrrrssssssttttttuuuuvvwxxxxxyzzzz'

In [38]:
random_wordsearch_letter()

'x'

In [39]:
def pad_grid(g0):
    grid = copy.deepcopy(g0)
    w = len(grid[0])
    h = len(grid)
    for r in range(h):
        for c in range(w):
            if grid[r][c] == '.':
                grid[r][c] = random_wordsearch_letter()
    return grid

In [53]:
padded = pad_grid(g)
print(show_grid(padded))

dexawwtmxepybossmezt
snaprhlukulelefaoder
ijreriotebahrevordla
jrsivcciprstdekcahkp
atvecshkoiselapcydcd
nesrehpargohtilljpue
nuleaveyisasmuguhhrt
eyhsifntzrmnsaeesetc
xverbseebasedlritlla
prcckzwiefamdonaiaeg
osacqnsofgadmeeulsoo
gdoporcmanaotpqiinmp
ncmusbaslpnteaerdkiu
ionjoyswleirinioicns
wnkcentcskeracllkase
oabsuerusopxeaodantt
dureifidimuhedgnskea
agagvccapitolsgyscrl
hkvgsandblasterdhihi
syenesdratoelhitsnod


In [54]:
print(show_grid(g))

dexawwt....ybossm..t
snaprhlukulele..o.er
...erioteb.hrevordla
..sivc..pr.tdekcahkp
atve..h.oiselapcy.cd
nesrehpargohtill.pue
n.leaveyis.smuguhhrt
eyhsifnt.r...aeesetc
xverbseebasedlritl.a
.r..k.wie..mdonaiaeg
..ac.nsof.admeeulsoo
g.opo.cmanaotpqiinmp
nc.us.a.lpnteaerdkiu
ionjoys.leirinioicns
wnkcent.skeracllkase
oab..erusopxeao.antt
dureifidimuhed.nskea
aga...capitols..scrl
h.v.sandblaster..i.i
s.e..sdratoelhitsn.d


In [55]:
for w in ws:
    print(w, present(padded, w))

newscast (True, 7, 6, <Direction.down: 4>)
kittenish (True, 14, 9, <Direction.upright: 6>)
cocks (True, 12, 1, <Direction.upright: 6>)
lithographers (True, 5, 14, <Direction.left: 1>)
truckle (True, 7, 18, <Direction.up: 3>)
leotards (True, 19, 12, <Direction.left: 1>)
they (True, 3, 11, <Direction.up: 3>)
exposure (True, 15, 12, <Direction.left: 1>)
dehumidifier (True, 16, 13, <Direction.left: 1>)
sandblaster (True, 18, 4, <Direction.right: 2>)
alien (True, 9, 17, <Direction.downleft: 7>)
paddle (True, 12, 9, <Direction.upright: 6>)
shadowing (True, 19, 0, <Direction.up: 3>)
gondola (True, 9, 19, <Direction.downleft: 7>)
wrest (True, 0, 5, <Direction.downleft: 7>)
joys (True, 13, 3, <Direction.right: 2>)
minster (True, 11, 18, <Direction.down: 4>)
pales (True, 4, 14, <Direction.left: 1>)
chairs (True, 3, 5, <Direction.downright: 8>)
fishy (True, 7, 5, <Direction.left: 1>)
capitols (True, 17, 6, <Direction.right: 2>)
based (True, 8, 8, <Direction.right: 2>)
gums (True, 6, 14, <Directio

In [43]:
def decoys(grid, words, all_words, limit=100):
    decoy_words = []
    dlen_limit = max(len(w) for w in words)
    while len(words) + len(decoy_words) < limit:
        d = random.choice(all_words)
        if d not in words and len(d) >= 4 and len(d) <= dlen_limit and not present(grid, d)[0]:
            decoy_words += [d]
    return decoy_words

In [44]:
ds = decoys(padded, ws, ws_words)
ds

['friendship',
 'stepping',
 'featheriest',
 'thriftily',
 'mutilation',
 'nook',
 'clewing',
 'meditated',
 'gooier',
 'cripples',
 'ponderously',
 'roundelay',
 'curtailed',
 'redeemed',
 'perimeters',
 'harelips',
 'overcompensating',
 'rejoicings',
 'adobe',
 'decreasing',
 'interstices',
 'curd',
 'orientate',
 'blueberries',
 'juniors',
 'broadloom',
 'debarring',
 'chandeliers',
 'segues',
 'army',
 'snuck',
 'pugilistic',
 'snugs',
 'dexterity',
 'dallies',
 'curving',
 'newsletter',
 'torn',
 'beaching',
 'limit',
 'blackguards',
 'breezier',
 'reoccur',
 'cabins']

In [45]:
for w in ws + ds:
    print(w, present(padded, w))

opted (True, 5, 7, <Direction.downright: 8>)
haywire (True, 8, 9, <Direction.downleft: 7>)
busting (True, 4, 17, <Direction.left: 1>)
succinct (True, 7, 17, <Direction.down: 4>)
institute (True, 16, 18, <Direction.up: 3>)
illicit (True, 13, 14, <Direction.left: 1>)
hypersensitivity (True, 16, 15, <Direction.left: 1>)
placement (True, 3, 11, <Direction.left: 1>)
bathrobe (True, 0, 5, <Direction.right: 2>)
inured (True, 4, 0, <Direction.right: 2>)
caveats (True, 3, 8, <Direction.downleft: 7>)
revisiting (True, 2, 11, <Direction.left: 1>)
pulp (True, 15, 11, <Direction.right: 2>)
teacup (True, 7, 16, <Direction.downleft: 7>)
threading (True, 18, 6, <Direction.right: 2>)
queered (True, 5, 13, <Direction.right: 2>)
parking (True, 6, 9, <Direction.right: 2>)
advent (True, 1, 11, <Direction.left: 1>)
chasuble (True, 19, 19, <Direction.left: 1>)
mosey (True, 7, 5, <Direction.downleft: 7>)
highboys (True, 19, 4, <Direction.right: 2>)
recharging (True, 18, 19, <Direction.up: 3>)
flue (True, 12, 

In [46]:
g, ws = interesting_grid()
p = pad_grid(g)
ds = decoys(p, ws, ws_words)
print(show_grid(g))
print(show_grid(p))
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
print('Present:', wcat(sorted(ws)))
print('Decoys:', wcat(sorted(ds)))

s.esevresed...dwhims
e.tbk..vapourse.bn.d
h.ificembracesnslo.e
ctr.egukiwic..ddui.r
inor.dwh.rips.rrftsa
reue.eeisrar.tiefiyl
mmli.mgrg.muarth.slc
ieft.un.a.bnbuemdole
nn.nesimrliertseepad
ei.imeloeccdeh.epob.
dfsaprlrio.saf.smri.
cnpdt.ofrn.usu..ap.h
oom.ispeccgntlpew.sa
tcu.e.l.lu.eda.vsgin
e.bdsb.oarrmneloplsg
r.olaetrleromrkr.isa
ibatnhd.nlpoaeic.bir
eesiee.i.luepno.o.e.
snt.d.o.y.pcte.p.mr.
u....jtoquesylduol..
sfesevresedpzcdwhims
eotbkvgvapoursehbnyd
hiificembracesnslone
ctrnegukiwicurdduivr
inorydwhrripscrrftsa
reueleeisrarvtiefiyl
mmlinmgrgzmuarthgslc
ieftuunyanbnbuemdole
nncnesimrliertseepad
eirimeloeccdehzepobm
dfsaprlrioisafesmriq
cnpdtsofrnausuodapxh
oomlispeccgntlpewasa
tcuaehlzluledakvsgin
eibdsbeoarrmneloplsg
rbolaetrleromrkrnisa
ibatnhdtnlpoaeicibir
eesieerimluepnoholey
sntadvoaycpctespsmro
uamapjtoquesylduoldp
56 words added;  8 directions
Present: abreast bigwig bluff bodes bumps clothe concur confinement coteries crier cull daintier declared dendrites denim deserves

In [63]:
g, ws = interesting_grid(width=10, height=10, words_limit=5, direction_slack=6)
p = pad_grid(g)
ds = decoys(p, ws, ws_words)
print(show_grid(g))
print()
print(show_grid(p))
print(len(ws), 'words added; ', len(set(present(g, w)[3] for w in ws)), 'directions')
print('Present:', wcat(sorted(ws)))
print('Decoys:', wcat(sorted(ds)))

...p.mown.
.sdse..ee.
.e.elad.cr
pi.dtir.ah
rzsiwovspu
oawh.kieab
brow.c.rda
ecnotops.r
d.kc.d...b
.staple...

fhjpamownq
wsdseuqeev
ieaeladhcr
piedtiriah
rzsiwovspu
oawhakieab
browpcfrda
ecnotopssr
dikchdnpnb
bstapleokr
14 words added;  6 directions
Present: apace cowhides crazies dock knows lived mown pears probed rhubarb rioted staple tops wide
Decoys: adapting bombing boor brick cackles carnal casino chaplets chump coaster coccyxes coddle collies creels crumbled cunt curds curled curlier deepen demeanor dicier dowses ensuing faddish fest fickler foaming gambol garoting gliding gristle grunts guts ibex impugns instants kielbasy lanyard loamier lugs market meanly minuend misprint mitts molested moonshot mucking oaks olives orgasmic pastrami perfect proceed puckered quashed refined regards retraces revel ridges ringlet scoff shinier siren solaria sprain sunder sunup tamped tapes thirds throw tiller times trains tranquil transfix typesets uric wariness welts whimsy winced winced


In [98]:
ds_original = ['regards', 'perfect', 'instants', 'refined', 'coddle', 'fickler', 'gambol', 'misprint', 'tapes', 'impugns', 'moonshot', 'chump', 'brick', 'siren', 'faddish', 'winced', 'kielbasy', 'market', 'puckered', 'trains', 'welts', 'cackles', 'foaming', 'proceed', 'gliding', 'guts', 'uric', 'oaks', 'molested', 'curled', 'boor', 'solaria', 'gristle', 'bombing', 'loamier', 'ensuing', 'cunt', 'sunder', 'revel', 'coaster', 'grunts', 'mucking', 'typesets', 'carnal', 'whimsy', 'scoff', 'coccyxes', 'meanly', 'sprain', 'minuend', 'ringlet', 'fest', 'winced', 'shinier', 'dicier', 'thirds', 'olives', 'garoting', 'pastrami', 'tranquil', 'tamped', 'sunup', 'crumbled', 'throw', 'ridges', 'chaplets', 'curlier', 'lugs', 'collies', 'adapting', 'demeanor', 'deepen', 'lanyard', 'tiller', 'transfix', 'wariness', 'times', 'mitts', 'dowses', 'creels', 'curds', 'quashed', 'orgasmic', 'ibex', 'retraces', 'casino']
ds = random.sample(ds_original, 15)
ds

['fickler',
 'adapting',
 'chump',
 'foaming',
 'molested',
 'carnal',
 'crumbled',
 'guts',
 'minuend',
 'bombing',
 'winced',
 'coccyxes',
 'solaria',
 'shinier',
 'cackles']

In [89]:
ds_original

['regards',
 'perfect',
 'instants',
 'refined',
 'coddle',
 'fickler',
 'gambol',
 'misprint',
 'tapes',
 'impugns',
 'moonshot',
 'chump',
 'brick',
 'siren',
 'faddish',
 'winced',
 'kielbasy',
 'market',
 'puckered',
 'trains',
 'welts',
 'cackles',
 'foaming',
 'proceed',
 'gliding',
 'guts',
 'uric',
 'oaks',
 'molested',
 'curled',
 'boor',
 'solaria',
 'gristle',
 'bombing',
 'loamier',
 'ensuing',
 'cunt',
 'sunder',
 'revel',
 'coaster',
 'grunts',
 'mucking',
 'typesets',
 'carnal',
 'whimsy',
 'scoff',
 'coccyxes',
 'meanly',
 'sprain',
 'minuend',
 'ringlet',
 'fest',
 'winced',
 'shinier',
 'dicier',
 'thirds',
 'olives',
 'garoting',
 'pastrami',
 'tranquil',
 'tamped',
 'sunup',
 'crumbled',
 'throw',
 'ridges',
 'chaplets',
 'curlier',
 'lugs',
 'collies',
 'adapting',
 'demeanor',
 'deepen',
 'lanyard',
 'tiller',
 'transfix',
 'wariness',
 'times',
 'mitts',
 'dowses',
 'creels',
 'curds',
 'quashed',
 'orgasmic',
 'ibex',
 'retraces',
 'casino']

In [99]:
print('grid = ', g)
print('padded_grid = ', p)
print('present_words = ', ws)
print('decoy_words = ', ds)
print('Directions: ', [(w, '`' + str(present(g, w)) + '`') for w in ws])

grid =  [['.', '.', '.', 'p', '.', 'm', 'o', 'w', 'n', '.'], ['.', 's', 'd', 's', 'e', '.', '.', 'e', 'e', '.'], ['.', 'e', '.', 'e', 'l', 'a', 'd', '.', 'c', 'r'], ['p', 'i', '.', 'd', 't', 'i', 'r', '.', 'a', 'h'], ['r', 'z', 's', 'i', 'w', 'o', 'v', 's', 'p', 'u'], ['o', 'a', 'w', 'h', '.', 'k', 'i', 'e', 'a', 'b'], ['b', 'r', 'o', 'w', '.', 'c', '.', 'r', 'd', 'a'], ['e', 'c', 'n', 'o', 't', 'o', 'p', 's', '.', 'r'], ['d', '.', 'k', 'c', '.', 'd', '.', '.', '.', 'b'], ['.', 's', 't', 'a', 'p', 'l', 'e', '.', '.', '.']]
padded_grid =  [['f', 'h', 'j', 'p', 'a', 'm', 'o', 'w', 'n', 'q'], ['w', 's', 'd', 's', 'e', 'u', 'q', 'e', 'e', 'v'], ['i', 'e', 'a', 'e', 'l', 'a', 'd', 'h', 'c', 'r'], ['p', 'i', 'e', 'd', 't', 'i', 'r', 'i', 'a', 'h'], ['r', 'z', 's', 'i', 'w', 'o', 'v', 's', 'p', 'u'], ['o', 'a', 'w', 'h', 'a', 'k', 'i', 'e', 'a', 'b'], ['b', 'r', 'o', 'w', 'p', 'c', 'f', 'r', 'd', 'a'], ['e', 'c', 'n', 'o', 't', 'o', 'p', 's', 's', 'r'], ['d', 'i', 'k', 'c', 'h', 'd', 'n', 'p'

In [100]:
# with open('example-wordsearch.txt', 'w') as f:
#     f.write('10x10\n')
#     f.write(show_grid(p))
#     f.write('\n')
#     f.write(lcat(sorted(ws + ds)))
# with open('exmaple-wordsearch-solution.txt', 'w') as f:
#     f.write('10x10\n')
#     f.write(show_grid(g))
#     f.write('\n')
#     f.write(lcat(sorted(ws)) + '\n\n')
#     f.write(lcat(sorted(ds)))

In [77]:
cts = collections.Counter()
for w in ws:
    _, r, c, d = present(g, w)
    inds = indices(g, r, c, len(w), d)
    for i in inds:
        cts[i] += 1
    print(w, r, c, len(w), d, inds)
[i for i in cts if cts[i] > 1]

probed 3 0 6 Direction.down [(3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0)]
staple 9 1 6 Direction.right [(9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6)]
rioted 6 7 6 Direction.upleft [(6, 7), (5, 6), (4, 5), (3, 4), (2, 3), (1, 2)]
cowhides 8 3 8 Direction.up [(8, 3), (7, 3), (6, 3), (5, 3), (4, 3), (3, 3), (2, 3), (1, 3)]
tops 7 4 4 Direction.right [(7, 4), (7, 5), (7, 6), (7, 7)]
knows 8 2 5 Direction.up [(8, 2), (7, 2), (6, 2), (5, 2), (4, 2)]
lived 2 4 5 Direction.downright [(2, 4), (3, 5), (4, 6), (5, 7), (6, 8)]
rhubarb 2 9 7 Direction.down [(2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9)]
crazies 7 1 7 Direction.up [(7, 1), (6, 1), (5, 1), (4, 1), (3, 1), (2, 1), (1, 1)]
dock 8 5 4 Direction.up [(8, 5), (7, 5), (6, 5), (5, 5)]
apace 5 8 5 Direction.up [(5, 8), (4, 8), (3, 8), (2, 8), (1, 8)]
mown 0 5 4 Direction.right [(0, 5), (0, 6), (0, 7), (0, 8)]
pears 0 3 5 Direction.downright [(0, 3), (1, 4), (2, 5), (3, 6), (4, 7)]
wide 4 4 4 Direction.upright [(4, 4), (3, 5), (2

[(7, 5), (2, 3), (3, 5)]

In [143]:
# for i in range(100):
#     print(i)
#     g, ws = interesting_grid()
#     p = pad_grid(g)
#     ds = decoys(p, ws, ws_words)
#     with open('wordsearch{:02}.txt'.format(i), 'w') as f:
#         f.write('20x20\n')
#         f.write(show_grid(p))
#         f.write('\n')
#         f.write(lcat(sorted(ws + ds)))
#     with open('wordsearch-solution{:02}.txt'.format(i), 'w') as f:
#         f.write('20x20\n')
#         f.write(show_grid(g))
#         f.write('\n')
#         f.write(lcat(sorted(ws)) + '\n\n')
#         f.write(lcat(sorted(ds)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
